1. Descarreguem les llibreries necessàries


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.feature_selection import f_classif
from tensorflow.keras.optimizers import SGD

2. Ens connectem al drive

In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


3. Ens descarreguem i insertem el fitxer .csv amb les dades

In [ ]:
df= pd.read_csv("/content/drive/MyDrive/DADES_TFM_SECTORS.csv",sep=';', encoding='ISO-8859-1')
df.shape

(1032318, 11)

4. Declarem les variables d'entrada i la variable de sortida

In [ ]:
x = df[['Any observat ','Antiguitat','length','matcat_id','pnom','dnom','Fallades anteriors','Fallades anteriors en el mateix sector']]
y = df[['fallada']]

5. Declarem la variable de divisió per obtenir les dades d'entrenament i validació

In [ ]:
campo_division = 'Any observat '

6. Considerem les dades d'entrenament, les que tenen l'any observat del 2005 fins al 2015

In [ ]:
x_entrenamiento = x[x['Any observat ']< 2016]
y_entrenamiento = y[y.index.isin(x_entrenamiento.index)]

7. Per un altre costat, les dades de validació van del 2016 al 202

In [ ]:
x_validacion = x[x['Any observat '] >= 2016]
y_validacion = y[y.index.isin(x_validacion.index)]

8. Tornem a delcarar les variables x, ara sense incorporar la variable d'any observat, ja que no la volem en l'estudi

In [ ]:
columnas_seleccionadas = ['Antiguitat','length','matcat_id','pnom','dnom','Fallades anteriors','Fallades anteriors en el mateix sector']
x_def = x[columnas_seleccionadas]
x_validacion_def = x_validacion[columnas_seleccionadas]
x_entrenamiento_def = x_entrenamiento[columnas_seleccionadas]

9. Estandaritzem les dades d'entrada

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_entrenamiento_def = scaler.fit_transform(x_entrenamiento_def)
x_validacion_def = scaler.transform(x_validacion_def)

10. Declarem el model construït

In [ ]:
modelo = keras.Sequential()
modelo.add(keras.layers.Dense(units=128,activation='relu', input_shape=(x_entrenamiento_def.shape[1],)),)
modelo.add(keras.layers.Dropout(rate=0.3))
modelo.add(keras.layers.Dense(units=128,activation='relu'))
modelo.add(keras.layers.Dropout(rate=0.3))
modelo.add(keras.layers.Dense(units=1,activation='sigmoid'))

11. Compilem el model, i informem del optimitzador, la pèrdua i la métrica a avaluar a cada pas

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.0001)
modelo.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

12. Entrenem el model

In [ ]:
historia = modelo.fit(x_entrenamiento_def, y_entrenamiento, epochs= 20, batch_size=32, validation_split=0.2)

Epoch 1/20
2982/2982 [==============================] - 11s 3ms/step - loss: 0.0701 - accuracy: 0.9910 - val_loss: 0.0551 - val_accuracy: 0.9907
Epoch 2/20
2982/2982 [==============================] - 9s 3ms/step - loss: 0.0368 - accuracy: 0.9938 - val_loss: 0.0536 - val_accuracy: 0.9907
Epoch 3/20
2982/2982 [==============================] - 8s 3ms/step - loss: 0.0358 - accuracy: 0.9938 - val_loss: 0.0521 - val_accuracy: 0.9907
Epoch 4/20
2982/2982 [==============================] - 9s 3ms/step - loss: 0.0351 - accuracy: 0.9938 - val_loss: 0.0517 - val_accuracy: 0.9907
Epoch 5/20
2982/2982 [==============================] - 8s 3ms/step - loss: 0.0350 - accuracy: 0.9939 - val_loss: 0.0522 - val_accuracy: 0.9907
Epoch 6/20
2982/2982 [==============================] - 12s 4ms/step - loss: 0.0346 - accuracy: 0.9938 - val_loss: 0.0518 - val_accuracy: 0.9907
Epoch 7/20
2982/2982 [==============================] - 8s 3ms/step - loss: 0.0342 - accuracy: 0.9939 - val_loss: 0.0517 - val_accurac

13. Realitzem una predicció amb les dades de validació

In [ ]:
y_pred_prob = modelo.predict(x_validacion_def)
y_pred_classes = np.round(y_pred_prob).flatten()

1917/1917 [==============================] - 3s 1ms/step


14. Comprobem la importància de les variables d'entrada durant l'entrenament

In [ ]:
weights_input_layer = modelo.layers[0].get_weights()[0]
from sklearn.feature_selection import f_classif
f_values, _ = f_classif(x_entrenamiento_def, y_entrenamiento)
importance_df = pd.DataFrame({'Variable': x_def.columns[:9], 'Importancia': f_values})
importance_df = importance_df.sort_values(by='Importancia', ascending=False)
print(importance_df)

                                 Variable  Importancia
5                      Fallades anteriors  1645.892883
1                                  length   924.647428
6  Fallades anteriors en el mateix sector   344.704361
3                                    pnom   103.053239
4                                    dnom    96.058518
0                              Antiguitat    61.113069
2                               matcat_id     0.020527


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


15. Declarem el llindar de decisió

In [ ]:
thresholds = [0.005]

16. Realitzem prediccions segons el llindar proposat

In [ ]:
for threshold in thresholds:
    y_pred = (y_pred_prob >= threshold).astype(int)
    y_pred_classes = np.round(y_pred).flatten()

17. Preparem els paràmetres de precicisó i matriu de confusió per avaluar el model

In [ ]:
accuracy = accuracy_score(y_validacion, y_pred_classes)
conf_matrix = confusion_matrix(y_validacion, y_pred_classes)
    print(f"Threshold: {threshold}")
    print(f"Accuracy: {accuracy:.2f}")
    print(conf_matrix)
    print("\n")

18. Preparem les variables de cada cel·la que conforma la matriu de confusió

In [ ]:
P=conf_matrix[1]
N=conf_matrix[0]
VP=P[1]
FN=P[0]
VN=N[0]
FP=N[1]

19. Declarem els tres paràmetres addicionals per avaluar el rendiment del model

In [ ]:
sensibilidad = VP / (VP + FN)
especificidad = VN / (VN + FP)
exactitud = (VP + VN) / conf_matrix.sum()
print("Sensibilidad:",sensibilidad)
print("Especificidad:" ,especificidad)
print("Exactitud:", exactitud)